# Three Node Network

Linear topology with three nodes and two BSM nodes for entanglement distribution

<img src="./notebook_images/teee.png" width="500"/>


The SeQUeNCe simulator makes use of the following modules:

- `Timeline`provides an interface for the discrete-event simulation kernel.
- `QuantumRouter` implements a quantum router. 
- `BSMNode` implements a quantum node for entanglement generation.
- `QuantumChannel` and `ClassicalChannel` serve as communication links between the nodes.

In [1]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import time

In [2]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QuantumRouter, BSMNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel

In [3]:
def test(sim_time, cc_delay, qc_atten, qc_dist):
    """
    sim_time: duration of simulation time (ms)
    cc_delay: delay on classical channels (ns)
    qc_atten: attenuation on quantum channels (db/m)
    qc_dist: distance of quantum channels (km)
    """
    cc_delay *= 1e6
    qc_dist *= 1e3
    raw_fidelity = 0.85
    tl = Timeline(sim_time * 1e9)
    tl.seed(0)
    
    r1 = QuantumRouter("r1", tl, 50)
    r2 = QuantumRouter("r2", tl, 100)
    r3 = QuantumRouter("r3", tl, 50)
    m1 = BSMNode("m1", tl, ["r1", "r2"])
    m2 = BSMNode("m2", tl, ["r2", "r3"])
    
    r1.add_bsm_node(m1.name, r2.name)
    r2.add_bsm_node(m1.name, r1.name)
    r2.add_bsm_node(m2.name, r3.name)
    r3.add_bsm_node(m2.name, r2.name)
    
    for node in [r1, r2, r3]:
        node.memory_array.update_memory_params("coherence_time", 10)
    for node in [r1,r2,r3]:
        node.memory_array.update_memory_params("raw_fidelity", raw_fidelity)
    
    nodes = [r1, r2, r3, m1, m2]
    for node1 in nodes:
        for node2 in nodes:
            if node1 == node2:
                continue
            cc = ClassicalChannel("cc_%s_%s"%(node1.name, node2.name), tl, 1e3, delay=cc_delay)
            cc.set_ends(node1, node2.name)
            
    for i, node in enumerate(nodes):  # Random Number Generation 
        node.set_seed(i)

    r1.network_manager.protocol_stack[0].add_forwarding_rule("r2", "r2")
    r1.network_manager.protocol_stack[0].add_forwarding_rule("r3", "r2")
    r2.network_manager.protocol_stack[0].add_forwarding_rule("r1", "r1")
    r2.network_manager.protocol_stack[0].add_forwarding_rule("r3", "r3")
    r3.network_manager.protocol_stack[0].add_forwarding_rule("r1", "r2")
    r3.network_manager.protocol_stack[0].add_forwarding_rule("r2", "r2")
    
    qc0 = QuantumChannel("qc_r1_m1", tl, qc_atten, qc_dist)
    qc1 = QuantumChannel("qc_r2_m1", tl, qc_atten, qc_dist)
    qc0.set_ends(r1, m1.name)
    qc1.set_ends(r2, m1.name)
    qc2 = QuantumChannel("qc_r2_m2", tl, qc_atten, qc_dist)
    qc3 = QuantumChannel("qc_r3_m2", tl, qc_atten, qc_dist)
    qc2.set_ends(r2, m2.name)
    qc3.set_ends(r3, m2.name)
    
    tl.init()
    
    r1.network_manager.request("r3", 1e12, 1e14, 50, 0.9)

    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    fig.set_size_inches(12, 5)

    data = []
    for info in r1.resource_manager.memory_manager:
        if info.entangle_time > 0:
            data.append(info.entangle_time / 1e12)
    data.sort()
    ax1.plot(data, range(1, len(data) + 1), marker="o")
    ax1.set_title("r1")
    ax1.set_ylabel("Number of Entangled Memories")
    
    data = []
    for info in r2.resource_manager.memory_manager:
        if info.entangle_time > 0:
            data.append(info.entangle_time / 1e12)
    data.sort()
    ax2.plot(data, range(1, len(data) + 1), marker="o")
    ax2.set_title("r2")
    ax2.set_xlabel("Simulation Time (s)")
    
    data = []
    for info in r3.resource_manager.memory_manager:
        if info.entangle_time > 0:
            data.append(info.entangle_time / 1e12)
    data.sort()
    ax3.plot(data, range(1, len(data) + 1), marker="o")
    ax3.set_title("r3")
    
    fig.tight_layout()    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    fig.set_size_inches(12, 5)
    
    data = []
    for info in r1.resource_manager.memory_manager:
        data.append(info.fidelity)
    ax1.bar(range(len(data)), data)
    ax1.plot([0, len(data)], [raw_fidelity, raw_fidelity], "k--")
    ax1.plot([0, len(data)], [0.9, 0.9], "k--")
    ax1.set_ylim(0.7,1)
    ax1.set_title("r1")
    ax1.set_ylabel("Fidelity")

    data = []
    for info in r2.resource_manager.memory_manager:
        data.append(info.fidelity)
    ax2.bar(range(len(data)), data)
    ax2.plot([0, len(data)], [raw_fidelity, raw_fidelity], "k--")
    ax2.plot([0, len(data)], [0.9, 0.9], "k--")
    ax2.set_ylim(0.7,1)
    ax2.set_title("r2")
    ax2.set_xlabel("Memory Number")

    data = []
    for info in r3.resource_manager.memory_manager:
        data.append(info.fidelity)
    ax3.bar(range(len(data)), data)
    ax3.plot([0, len(data)], [raw_fidelity, raw_fidelity], "k--")
    ax3.plot([0, len(data)], [0.9, 0.9], "k--")
    ax3.set_ylim(0.7,1)
    ax3.set_title("r3")
    
    fig.tight_layout()

### Execution

Parameters:

    sim_time: duration of simulation time (ms)
    cc_delay: delay on classical channels (ns)
    qc_atten: attenuation on quantum channels (db/m)
    qc_dist: distance of quantum channels (km)
    

In [4]:
interactive_plot = interact(test, sim_time=(2000, 4000, 500), cc_delay=(100, 1000, 100), qc_atten=[1e-5, 2e-5, 3e-5], qc_dist=(1, 10, 1))
interactive_plot

interactive(children=(IntSlider(value=3000, description='sim_time', max=4000, min=2000, step=500), IntSlider(v…

<function __main__.test(sim_time, cc_delay, qc_atten, qc_dist)>